# Customer Purchase Prediction

RetailTech Solutions is a fast-growing international e-commerce platform operating in over 20 countries across Europe, North America, and Asia. They specialize in fashion, electronics, and home goods, with a unique business model that combines traditional retail with a marketplace for independent sellers.

The company has seen rapid growth. A key part of their success has been their data-driven approach to personalization. However, as they plan their expansion into new markets, they need to improve their ability to predict customer behavior.

Their marketing team wants to predict which customers are most likely to make a purchase based on their browsing behavior.

As an AI Engineer, you will help build this prediction system. Your work will directly impact RetailTech's growth strategy and their goal of increasing revenue.


## Data Description

| Column Name | Criteria |
|------------|----------|
| customer_id | Integer. Unique identifier for each customer. No missing values. |
| time_spent | Float. Minutes spent on website per session. Missing values should be replaced with median. |
| pages_viewed | Integer. Number of pages viewed in session. Missing values should be replaced with mean. |
| basket_value | Float. Value of items in basket. Missing values should be replaced with 0. |
| device_type | String. One of: Mobile, Desktop, Tablet. Missing values should be replaced with "Unknown". |
| customer_type | String. One of: New, Returning. Missing values should be replaced with "New". |
| purchase | Binary. Whether customer made a purchase (1) or not (0). Target variable. |

# Task 1

The marketing team has collected customer session data in `raw_customer_data.csv`, but it contains missing values and inconsistencies that need to be addressed.
Create a cleaned version of the dataframe:

- Start with the data in the file `raw_customer_data.csv`
- Your output should be a DataFrame named `clean_data`
- All column names and values should match the table below.
</br>

| Column Name | Criteria |
|------------|----------|
| customer_id | Integer. Unique identifier for each customer. No missing values. |
| time_spent | Float. Minutes spent on website per session. Missing values should be replaced with median. |
| pages_viewed | Integer. Number of pages viewed in session. Missing values should be replaced with mean. |
| basket_value | Float. Value of items in basket. Missing values should be replaced with 0. |
| device_type | String. One of: Mobile, Desktop, Tablet. Missing values should be replaced with "Unknown". |
| customer_type | String. One of: New, Returning. Missing values should be replaced with "New". |
| purchase | Binary. Whether customer made a purchase (1) or not (0). Target variable. |

In [17]:
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv('raw_customer_data.csv')

# Step 2: Clean missing values
df['time_spent'].fillna(df['time_spent'].median(), inplace=True)
df['pages_viewed'].fillna(df['pages_viewed'].mean(), inplace=True)
df['basket_value'].fillna(0, inplace=True)
df['device_type'].fillna('Unknown', inplace=True)
df['customer_type'].fillna('New', inplace=True)

# Step 3: Ensure correct data types
df['customer_id'] = df['customer_id'].astype(int)
df['pages_viewed'] = df['pages_viewed'].astype(int)
df['purchase'] = df['purchase'].astype(int)

# Step 4: Store cleaned data
clean_data = df

# Task 2
The pre-cleaned dataset `model_data.csv` needs to be prepared for our neural network.
Create the model features:

- Start with the data in the file `model_data.csv`
- Scale numerical features (`time_spent`, `pages_viewed`, `basket_value`) to 0-1 range
- Apply one-hot encoding to the categorical features (`device_type`, `customer_type`)
    - The column names should have the following format: variable_name_category_name (e.g., `device_type_Desktop`)
- Your output should be a DataFrame named `model_feature_set`, with all column names from `model_data.csv` except for the columns where one-hot encoding was applied.


In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load the data
df = pd.read_csv('model_data.csv')

# Step 2: Scale numerical features
scaler = MinMaxScaler()
numerical_cols = ['time_spent', 'pages_viewed', 'basket_value']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Step 3: One-hot encode categorical features
categorical_cols = ['device_type', 'customer_type']
df_encoded = pd.get_dummies(df[categorical_cols], prefix=categorical_cols)

# Step 4: Drop original categorical columns and concatenate encoded columns
df.drop(columns=categorical_cols, inplace=True)
model_feature_set = pd.concat([df, df_encoded], axis=1)

# Task 3

Now that all preparatory work has been done, create and train a neural network that would allow the company to predict purchases.

- Using PyTorch, create a network with:
   - At least one hidden layer with 8 units
   - ReLU activation for hidden layer
   - Sigmoid activation for the output layer
- Using the prepared features in `input_model_features.csv`, train the model to predict purchases. 
- Use the validation dataset `validation_features.csv` to predict new values based on the trained model. 
- Your model should be named `purchase_model` and your output should be a DataFrame named `validation_predictions` with columns `customer_id` and `purchase`. The `purchase` column must be your predicted values.


In [19]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

# Step 1: Load the datasets
train_df = pd.read_csv('input_model_features.csv')
val_df = pd.read_csv('validation_features.csv')

# Step 2: Prepare features and labels
X_train = train_df.drop(['customer_id', 'purchase'], axis=1).values
y_train = train_df['purchase'].values

X_val = val_df.drop(['customer_id'], axis=1).values
val_customer_ids = val_df['customer_id'].values

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

# Step 3: Define the neural network
class PurchaseNet(nn.Module):
    def __init__(self, input_dim):
        super(PurchaseNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 8)
        self.relu = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.output(x))
        return x

# Initialize model
input_dim = X_train.shape[1]
purchase_model = PurchaseNet(input_dim)

# Step 4: Define loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(purchase_model.parameters(), lr=0.01)

# Step 5: Train the model
epochs = 100
for epoch in range(epochs):
    purchase_model.train()
    optimizer.zero_grad()
    outputs = purchase_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Step 6: Predict on validation data
purchase_model.eval()
with torch.no_grad():
    predictions = purchase_model(X_val_tensor)
    predicted_labels = (predictions.numpy() > 0.5).astype(int).flatten()

# Step 7: Create validation_predictions DataFrame
validation_predictions = pd.DataFrame({
    'customer_id': val_customer_ids,
    'purchase': predicted_labels
})